In [ ]:
#syooress CPU warn
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras as K
import matplotlib.pyplot as plt

In [ ]:
from numpy import genfromtxt
import pandas as pd
x = genfromtxt('/content/drive/MyDrive/Colab Notebooks/WISDM_x.csv', delimiter=',')
y = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/WISDM_y.csv').to_numpy()

In [ ]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
le = LabelEncoder()
y = le.fit_transform(y)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
def create_series(x,y,timestep,overlap):

  slide_step = int(timestep*(1-overlap))
  data_num = int((len(x)/slide_step)-1)
  
  dataset = np.ndarray(shape=(data_num,timestep,len(x[0])))
  labels = list()

  for i in range(data_num):
    labels.append(y[slide_step*(i+1)-1])
    for j in range(timestep):
      dataset[i,j,:] = x[slide_step*i+j,:]

  return dataset,np.array(labels)


In [ ]:
# y_pred.append("5")
#arrange 16 lines data into a series
#try to import time series in this way
import numpy as np
dataset,labels = create_series(x,y,50,0.2)
dataset = np.reshape(dataset,(len(dataset),len(dataset[0])*len(dataset[0,0]),1))

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(dataset,labels,test_size=0.3, random_state=412)
# X_test, X_val, y_test, y_val = train_test_split(X_toSplit,y_toSplit,test_size=0.25, random_state=412)

In [ ]:
x_train.shape[1]


150

In [ ]:
def trans_encoder(inputs, head_size, n_heads, ff_dim, drop=0.0):
  # MultiHeadAttention
  x = K.layers.LayerNormalization(epsilon=1e-6)(inputs)
  x = K.layers.MultiHeadAttention(
    key_dim=head_size, num_heads=n_heads, dropout=drop)(x, x)
  x = K.layers.Dropout(drop)(x)
  res = x + inputs

  x = K.layers.LayerNormalization(epsilon=1e-6)(res)
  x = K.layers.Conv1D(filters=ff_dim, kernel_size=1,
    activation="relu")(x)
  x = K.layers.Dropout(drop)(x)
  x = K.layers.Conv1D(filters=inputs.shape[-1],
    kernel_size=1)(x)
  return x + res

In [ ]:
def create_model(input_shape, head_size, n_heads, ff_dim,
  n_trans_blocks, mlp_units, drop=0.0, mlp_drop=0.0):

  inpts = K.Input(shape=input_shape)
  x = inpts
  for _ in range(n_trans_blocks):
    x = trans_encoder(x, head_size, n_heads, ff_dim, drop)

  x = K.layers.\
    GlobalAveragePooling1D(data_format="channels_first")(x)
  for dim in mlp_units:
    x = K.layers.Dense(dim, activation="relu")(x)
    x = K.layers.Dropout(mlp_drop)(x)
  oupts = K.layers.Dense(6, activation="softmax")(x) 
  return K.Model(inpts, oupts)

In [ ]:

  # 0. get ready
  print("\nBegin Transformer classification demo ")
  np.random.seed(1)
  tf.random.set_seed(1)
  print("Using Keras: " + str(K.__version__))

  # 1. load training and test data
  print("\nLoading Ford time series classification data ")
  data_root = "https://raw.githubusercontent.com/"
  data_root += "hfawaz/cd-diagram/master/FordA/"

  # x_train, y_train = load_xy(data_root + "FordA_TRAIN.tsv")
  # x_test, y_test = load_xy(data_root + "FordA_TEST.tsv")

  r = x_train.shape[0]
  c = x_train.shape[1]
  x_train = x_train.reshape((r, c, 1))  # (3601, 500, 1)

  r = x_test.shape[0]
  c = x_test.shape[1]
  x_test = x_test.reshape((r, c, 1))  # (1320, 500, 1)

  print("\nCreating Transformer model ")
  input_shape = x_train.shape[1:]  # (500,1)

  model = create_model(input_shape, head_size=256, n_heads=4,
    ff_dim=4, n_trans_blocks=4, mlp_units=[128],
    drop=0.25, mlp_drop=0.4)
  
  model.compile(loss="sparse_categorical_crossentropy",
    optimizer=K.optimizers.Adam(learning_rate=1.0e-4),
    metrics=["sparse_categorical_accuracy"])
  # c_backs = [K.callbacks.EarlyStopping(patience=10, \
  #   restore_best_weights=True)]

  print("\nStarting training ")
  model.fit(x_train, y_train, validation_split=0.2,
    epochs=300, batch_size=36, shuffle=True)    
  print("Training complete")

  print("\nSetting up inpt of 500 random values in (0,1) ")
  inpt = np.random.random((1,500,1))  # float64 will convert
  pred = model(inpt)
  print("Prediction: ")
  print(pred)

  print("\nEnd demo ")


Begin Transformer classification demo 
Using Keras: 2.9.0

Loading Ford time series classification data 

Creating Transformer model 

Starting training 
Epoch 1/300
408/408 [==============================] - 26s 57ms/step - loss: 4.1530 - sparse_categorical_accuracy: 0.4164 - val_loss: 1.6919 - val_sparse_categorical_accuracy: 0.5992
Epoch 2/300
408/408 [==============================] - 23s 56ms/step - loss: 2.1849 - sparse_categorical_accuracy: 0.5345 - val_loss: 1.1560 - val_sparse_categorical_accuracy: 0.6717
Epoch 3/300
408/408 [==============================] - 23s 55ms/step - loss: 1.5422 - sparse_categorical_accuracy: 0.5965 - val_loss: 0.9802 - val_sparse_categorical_accuracy: 0.7136
Epoch 4/300
408/408 [==============================] - 23s 55ms/step - loss: 1.2871 - sparse_categorical_accuracy: 0.6409 - val_loss: 0.8959 - val_sparse_categorical_accuracy: 0.7395
Epoch 5/300
408/408 [==============================] - 23s 56ms/step - loss: 1.1436 - sparse_categorical_accuracy

ValueError: ignored